In [73]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
pip install -U sentence-transformers

     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 2.0MB 11.5MB/s 
     |████████████████████████████████| 1.2MB 48.6MB/s 
     |████████████████████████████████| 890kB 49.0MB/s 
     |████████████████████████████████| 3.2MB 56.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.0.3-cp37-none-any.whl size=114277 sha256=646aa3fae60ee02f676fc236fd8576670ef4cdd4d4b18836318861ae8c579bb7
  Stored in directory: /root/.cache/pip/wheels/c8/0d/fa/4e28cf045da4781344e7972befb2fdf306051b225bfc290187
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=a4cbabc5874f3797cd29d0b1b6c8f0a94d5edf81d874c3228ae753bdd8379d15
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-distilroberta-base-v1')

In [ ]:
import json
import numpy as np
import os
import shutil
import pandas as pd
import nltk
import re
import pickle
import tensorflow as tf
import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding,Flatten,Dense,Bidirectional,LSTM,Dropout
from tensorflow.keras.models import Sequential
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import torch
from transformers import BertTokenizer, BertModel
import matplotlib.pyplot as plt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocess_text(string):
    string=string.lower()
    punctuations = '''!()[]{};:"\,<>/?@#$^&.*_~'''
    string=re.sub(r'[!?,.-=-{(}):;]+','',string)
    string=string.replace('.'," ")
    string=string.replace('\n'," ")
    string = re.sub(r'[^a-zA-Z0-9\']', ' ', string)
    string = ' '.join( [w for w in string.split() if (len(w)>1 or w is 'a')] )
    string = re.sub(' +', ' ', string) 
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, " ") 
    return string

In [ ]:
import pandas as pd
data1 = list(pd.read_csv('/content/drive/MyDrive/TECH MEET 2021/mobile_articles.csv')['Article'])
data2 = list(pd.read_csv('/content/drive/MyDrive/TECH MEET 2021/other_articles.csv')['Article'])

mobile = []
non_mobile=[]

for each in data1:
  mobile.append(preprocess_text(each))

for each in data2:
  non_mobile.append(preprocess_text(each))

import random

random.shuffle(mobile)
random.shuffle(non_mobile)

label1 = [1]*len(mobile)
label2 = [0]*len(non_mobile)

data = mobile + non_mobile

label = label1 + label2

df = pd.DataFrame(zip(data,label),columns=['text','label'])

from sklearn.utils import shuffle
df = shuffle(df)


from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2)
val, test = train_test_split(test, test_size=0.5)

X_train = list(train['text'])
Y_train = list(train['label'])
X_val = list(val['text'])
Y_val = list(val['label'])
X_test = list(test['text'])
Y_test = list(test['label'])


print(len(X_train))
print(len(X_val))
print(len(X_test))

print(len(Y_train))
print(len(Y_val))
print(len(Y_test))

3031
379
379
3031
379
379


In [ ]:
embed_train= model.encode(X_train)
embed_val=model.encode(X_val)
embed_test=model.encode(X_test)

embed_train=np.reshape(embed_train,(embed_train.shape[0],embed_train.shape[1],1))
embed_train = torch.tensor(embed_train)
embed_val=np.reshape(embed_val,(embed_val.shape[0],embed_val.shape[1],1))
embed_val = torch.tensor(embed_val)
embed_test=np.reshape(embed_test,(embed_test.shape[0],embed_test.shape[1],1))
embed_test=torch.tensor(embed_test)


Y_train = torch.tensor(Y_train)
Y_val = torch.tensor(Y_val)
Y_test=torch.tensor(Y_test)

print(embed_train.shape)
print(embed_val.shape)
print(embed_test.shape)
print(Y_train.shape)
print(Y_val.shape)
print(Y_test.shape)

torch.Size([3031, 768, 1])
torch.Size([379, 768, 1])
torch.Size([379, 768, 1])
torch.Size([3031])
torch.Size([379])
torch.Size([379])


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# Hyperparameters
input_size = 768
hidden_size = 640
num_layers = 2
num_classes = 2
#sequence_length = 28
#learning_rate = 0.005
#batch_size = 64
#num_epochs = 2

class TEXT_MODEL(nn.Module):
    
    def __init__(self, in_features, hidden_size, num_layers, out_features, bias = True):

        super(TEXT_MODEL, self).__init__()

        self.in_features = in_features
        self.out_features = out_features
       
        
        self.dense_1 = nn.Linear(768, 128)
        self.dense_2 = nn.Linear(128, 64)
        self.dense_3= nn.Linear(64, 32) #relu
        self.dropout = nn.Dropout(0.2)
        self.last_dense = nn.Linear(32, self.out_features) #sigmoid
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        """
        weight = torch.zeros(1, None,requires_grad=True)  #(768,1)
        nn.init.xavier_normal_(weight)
        self.weight = nn.Parameter(weight)
        """

    def forward(self, l):

        l = l.view(l.size(0), -1)
      
        l = self.relu(self.dense_1(l))
       
        l = self.relu(self.dense_2(l))
        l = self.relu(self.dense_3(l))
      
        l = self.dropout(l)
        model_output = self.sigmoid(self.last_dense(l))


        return model_output

In [ ]:
def multi_acc(y_pred, y_test):
    
    y_pred_m = (y_pred>0.5)
    y_pred_m = torch.tensor(y_pred_m,dtype=int)

    from sklearn.metrics import accuracy_score
    acc=accuracy_score(y_test,y_pred_m)
    
    return acc

x_train = embed_train.squeeze(dim=2)
x_val = embed_val.squeeze(dim=2)
y_train = Y_train.float().unsqueeze(dim=1)
y_val = Y_val.float().unsqueeze(dim=1)
text_model = TEXT_MODEL(768, 320, 2, 1)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(text_model.parameters(), lr=0.001)

In [ ]:
#FITTING AND COMPILING

for epoch in range(90):
    optimizer.zero_grad()
    out = text_model(x_train)
    out = out.float()
    #print(type(out), type(y_train))
    loss = criterion(out, y_train)
    loss.backward()
    optimizer.step()
    val_out = text_model(x_val)
    val_out = val_out.float()
    val_loss = criterion(val_out, y_val)


    print(f"Epoch {epoch+1}, loss: {loss.item()}, accuracy: {multi_acc(out, y_train)} val_loss: {val_loss.item()}, val_accuracy: {multi_acc(val_out, y_val)}")


Epoch 1, loss: 0.6939842104911804, accuracy: 0.45067634444077864 val_loss: 0.6818898916244507, val_accuracy: 0.8654353562005277


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  after removing the cwd from sys.path.


Epoch 2, loss: 0.6819620132446289, accuracy: 0.8495546024414384 val_loss: 0.671413242816925, val_accuracy: 0.8680738786279684
Epoch 3, loss: 0.6714098453521729, accuracy: 0.8663807324315408 val_loss: 0.6600127816200256, val_accuracy: 0.8707124010554089
Epoch 4, loss: 0.6602141857147217, accuracy: 0.8653909600791818 val_loss: 0.6467156410217285, val_accuracy: 0.8707124010554089
Epoch 5, loss: 0.6476646661758423, accuracy: 0.8657208841966347 val_loss: 0.6321409344673157, val_accuracy: 0.8707124010554089
Epoch 6, loss: 0.6327196359634399, accuracy: 0.8657208841966347 val_loss: 0.6135023832321167, val_accuracy: 0.8707124010554089
Epoch 7, loss: 0.6135969161987305, accuracy: 0.8657208841966347 val_loss: 0.5931942462921143, val_accuracy: 0.8707124010554089
Epoch 8, loss: 0.5919923782348633, accuracy: 0.8657208841966347 val_loss: 0.5674000978469849, val_accuracy: 0.8707124010554089
Epoch 9, loss: 0.567760705947876, accuracy: 0.8657208841966347 val_loss: 0.5370995998382568, val_accuracy: 0.870

In [81]:
with open("/content/drive/MyDrive/TECH MEET 2021/translated_tweets_abhi.json",'r') as out:
    t_a=json.load(out)

t_a=list(t_a.values())
#t_a=[preprocess_text(each) for each in t_a]

x_test=model.encode(t_a)

In [82]:
t_a

["lloydsbank how do report a scam on my account when don't do internet or telephone banking",
 'rt iameraijaz have been denied security in jammu by security agencies in jammu despite severe threats have been categorised by security agencies on basis of threatsthere has been increase in criminal attacks in jammu and can risk my lifekindly look into it hmoindia amitshah',
 "fplprop gooner fpl think there's a risk salah could be rested at least subbed before mins for city game if we're doing well against brighton",
 'rt brfootball official bayern munich loan defender chris richards to hoffenheim until the end of the season httpstcoxqjdgdfhw',
 'rt timinhonolulu so far three cycle up in are retiring senatorburr sentoomey robportman finally here are the cycle gop who are on the cusp should evidence emerge linking them to the attack they risk both indictment and push to remove them at the ballot box',
 'rt kaykmtz no one thought to make restaurant workers eligible for the vaccine before they

In [83]:
x_test = torch.tensor(x_test)
print(x_test.size())

torch.Size([210, 768])


In [84]:

with torch.no_grad():
    text_model.eval()
    y_pred = text_model(x_test)
    y_pred=y_pred>0.5

In [85]:
y_pred=y_pred.numpy().astype(int)

In [86]:
y_pred=np.reshape(y_pred,(210,))

In [87]:
print(y_pred)

[0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [88]:
import csv 
import pandas as pd
import time
df = pd.read_excel("/content/drive/MyDrive/TECH MEET 2021/evaluation_data.xlsx")

data=df['Text']

with open("TWEETS_CLASSIFIED12345.csv",'w') as out:
    writer=csv.writer(out)

    writer.writerow(['TWEETS','MOBILE TECH OR NOT'])

    for i,y in enumerate(y_pred):
        writer.writerow([t_a[i],y])
    

In [ ]:
t_a[0]

'market highlights an automotive constant velocity joint helps transmit torque to front or rear axles without friction at a constant speed constant velocity joint provide hasslefree ride without vibration noise constant velocity joint are surrounded by rubber which is filled with molybdenum disulfide grease which reduces friction wear increase in the per capita disposable income and growing purchasing power of the customers will enhance the growth of the market rapid increase in sales of passenger cars hatchback suv and sedans across the globe driving the growth of constant velocity joints growing construction industry throughout the globe increases the growth of constant velocity joint market due to the growing demand for heavy commercial vehicle increased logistics in place across the globe will also drive the global automotive constant velocity joint market moreover the growth of the market is challenged by the high prices of the parts such as half shafts axles and reduction of gree